In [1]:
# from csv import writer
import csv

# Comunicação com o MongoDB
import pymongo

# Data
from datetime import datetime

import pandas as pd
import re

import warnings
warnings.filterwarnings("ignore")

## Funções Auxiliares

In [2]:
# Função "Explode" na coluna Endereço
def explode_endereco(str):
    padroes = [
        r'Cond[a-z].+SP',
        r'Ru[a-z].+SP',
        r'Av[a-z].+SP',
        r'Praç[a-z].+SP',
        r'Alamed[a-z].+SP',
        r'Travess[a-z].+SP',
        r'Parqu[a-z].+SP',
        r'Viel[a-z].+SP'
    ]

    for padrao in padroes:
        resultado = re.findall(padrao, str)
        if resultado:
            return resultado[0].replace("', ' ", "-").replace('-', ' - ')

    # Se nenhum padrão foi encontrado, retorna None
    return None

## Explode Infos
# Andares
def andares(str):
    # Andares
    regex = r'[0-9]º\s[a-z]+\s[a-z]+\s[a-z]+'
    request = re.findall(regex, str)
    if request:
        return request[0]

    regex = r'Entre\s[0-9]º\sa\s[0-9]º\s\w+'
    request = re.findall(regex, str)
    if request:
        return request[0]

    floor_regex = r'[0-9]º\sandar'
    request = re.findall(floor_regex, str)
    if request:
        return request[0]

    return 'Sem info'

# Tamanho
def tamanho(str):
    regex = r'[0-9]+\sm²'
    request = re.findall(regex, str)
    if request:
        return request[0]

    return 'Sem info'

#Preco
def preco(str): 
    regex = r'R\$\s[0-9]+\.[0-9]+\s\/m²|R\$\s[0-9]+\s\/m²'
    request = re.findall(regex, str)
    if request:
        return request[0]

    return 'Sem info'

# Quartos
def quartos(str):
    regex = r'[0-9]\squarto[s]?'
    request = re.findall(regex, str)
    if request:
        return request[0]

    regex = r'studio'
    request = re.findall(regex, str)
    if request:
        return request[0]

    return 'Sem info'

# Suites
def suite(str):
    regex = r'[0-9]\ssuíte'
    request = re.findall(regex, str)
    if request: 
        return request[0]

    regex = r'sem\ssuíte'
    request = re.findall(regex, str)
    if request: 
        return request[0]
    
    return 'Sem info'

# Garagem
def garagem(str):
    regex = r'Não\s\-\sVag\w+'
    request = re.findall(regex, str)
    if request:
        return request[0]
    
    regex = r'[0-9]\svag\w+'
    request = re.findall(regex, str)
    if request: 
        return request[0]
    
    return 'Sem info'
    
# Banheiros
def banheiros(str):
    regex = r'[0-9]\sBan\w+'
    request = re.findall(regex, str)
    if request: 
        return request[0]
    
    return 'Sem banheiro'

# Varanda
def varanda(str):
    regex = r'[0-9]\sVar\w+s'
    request = re.findall(regex, str)
    if request:
        return request[0]

    regex = r'Var\w+a'
    request = re.findall(regex, str)
    if request:
        return request[0]
    
    return 'Sem varanda'

# Mobiliado
def mobiliado(str):
    regex = r'Não\s\-\sMob\w+'
    request = re.findall(regex, str)
    if request:
        return 'Não mobiliado'
    
    regex = r'Mob\w+'
    request = re.findall(regex, str)
    if request:
        return 'Mobiliado'
    
# Portaria
def portaria(str):
    regex = r'Por\w+'
    request = re.findall(regex, str)
    if request: 
        return request[0]
    
    return 'Sem info'

# Metro
def metro(str):
    regex = r'Metrô'
    request = re.findall(regex, str)
    if request: 
        regex = r'a\s[0-9]+m'
        request = re.findall(regex, str)
        return 'Metro {}'.format(request[0])

    else: 
        return 'Sem Metro próximo'

## Importando o .csv

In [61]:
df = pd.read_csv('C:/Users/lucas/Desktop/WebScraping/arquivo_teste/webscraping01.csv', sep=';', encoding='utf-8')

# Ajustando espaçamento das colunas
# print('Maior nº de caracteres utilizado nos dados: {}'.format(df['Informacoes'].str.len().max()))
pd.set_option('max_colwidth', 200)

# Inserindo data 
# today_date = datetime.now().date().strftime('%d/%m/%Y')
# df['Refdate'] = today_date
df['Refdate'] = '17/04/2023'
df = df[['Refdate', 'Link_Apto', 'Endereco', 'Bairro', 'Valor', 'Informacoes']]
df.head()

,Refdate,Link_Apto,Endereco,Bairro,Valor,Informacoes
0,17/04/2023,https://loft.com.br/imovel/apartamento-rua-caure-tucuruvi-sao-paulo-2-quartos-50m2/100tsld,"['Apartamento à venda em Tucuruvi com 50 m²', ' 2 quartos', ' 1 vagaCondomínio Vila Verde', ' Rua Cauré', ' Tucuruvi', ' São Paulo-SP']",Tucuruvi,R$ 325.000,"['Andar baixoAté 3º andar', '50 m²R$ 6.500 /m²', '2 quartossem suíte', '1 vaga', '1 Banheiro', 'Portaria24h', 'Não - Mobiliado']"
1,17/04/2023,https://loft.com.br/imovel/apartamento-rua-do-bosque-bom-retiro-sao-paulo-2-quartos-72m2/1014uwp,"['Apartamento à venda em Bom Retiro com 72 m²', ' 2 quartosCondomínio Edificio Palhinha', ' Rua do Bosque', ' Bom Retiro', ' São Paulo-SP']",BomRetiro,R$ 479.000,"['Andar alto4º andar em diante', '72 m²R$ 6.652 /m²', '2 quartossem suíte', '1 Banheiro', 'Metrô próximoestação a 885m', 'Não - Vagas', 'Não - Mobiliado']"
2,17/04/2023,https://loft.com.br/imovel/apartamento-rua-jose-maria-lisboa-jardim-america-sao-paulo-3-quartos-137m2/101r6vc,"['Apartamento à venda em Jardim América com 137 m²', ' 3 quartos', ' 1 suíteCondomínio Anambé', ' Rua José Maria Lisboa', ' Jardim América', ' São Paulo-SP']",JardimAmérica,R$ 1.450.000,"['Andar alto4º andar em diante', '137 m²R$ 10.583 /m²', '3 quartos1 suíte', '3 Banheiros', 'Metrô próximoestação a 702m', 'Não - Vagas', 'Não - Mobiliado']"
3,17/04/2023,https://loft.com.br/imovel/apartamento-avenida-doutor-cardoso-de-melo-vila-olimpia-sao-paulo-1-quarto-72m2/1024379,"['Apartamento à venda em Vila Olímpia com 72 m²', ' 1 quarto', ' 1 suíte', ' 2 vagasCondomínio Maxhaus Vila Olimpia', ' Avenida Doutor Cardoso de Melo', ' Vila Olímpia', ' São Paulo-SP']",VilaOlímpia,R$ 1.400.000,"['Andar alto4º andar em diante', '72 m²R$ 19.444 /m²', '1 quarto1 suíte', '2 vagas', '2 Banheiros', 'Varanda', 'Não - Mobiliado']"
4,17/04/2023,https://loft.com.br/imovel/apartamento-rua-nunes-garcia-santana-sao-paulo-4-quartos-200m2/102ww3k,"['Apartamento à venda em Santana com 200 m²', ' 4 quartos', ' 1 suíte', ' 3 vagasCondomínio Conjunto Santana Edificio Solar De Coimbra', ' Rua Nunes Garcia', ' Santana', ' São Paulo-SP']",Santana,R$ 850.000,"['Andar alto4º andar em diante', '200 m²R$ 4.250 /m²', '4 quartos1 suíte', '3 vagas', '4 Banheiros', 'Metrô próximoestação a 519m', 'Não - Mobiliado']"


## Arrumando as colunas

### - Aplicando <font color='yellow'>Regex</font>

In [62]:
# Arrumando o nome dos Bairro 
# De: VilaGuiomar Para: Vila Guiomar
df['Bairro'] = df['Bairro'].apply(lambda x: re.sub(r'(?<!^)(?=[A-Z])', ' ', x))

In [63]:
# Explode Endereço
df['Endereco'] = df['Endereco'].apply(lambda x: explode_endereco(x))

In [64]:
# Andares
df['Andares'] = df['Informacoes'].apply(lambda x: andares(x))

# Tamanho
df['Tamanho (m2)'] = df['Informacoes'].apply(lambda x: tamanho(x))

# Valor p/ m2
df['Valor (m2)'] = df['Informacoes'].apply(lambda x: preco(x))

# Quantidade de Quartos
df['Quartos'] = df['Informacoes'].apply(lambda x: quartos(x))

# Suites
df['Suites'] = df['Informacoes'].apply(lambda x: suite(x))

# Garagem
df['Garagem'] = df['Informacoes'].apply(lambda x: garagem(x))

# Banheiro
df['Banheiros'] = df['Informacoes'].apply(lambda x: banheiros(x))

# Varanda
df['Varanda'] = df['Informacoes'].apply(lambda x: varanda(x))

# Mobiliado
df['Mobiliado'] = df['Informacoes'].apply(lambda x: mobiliado(x))

# Portaria
df['Portaria'] = df['Informacoes'].apply(lambda x: portaria(x))

# Metro
df['Metro'] = df['Informacoes'].apply(lambda x: metro(x))

In [65]:
df.drop(['Informacoes'], axis=1, inplace= True)

In [66]:
df.head()

,Refdate,Link_Apto,Endereco,Bairro,Valor,Andares,Tamanho (m2),Valor (m2),Quartos,Suites,Garagem,Banheiros,Varanda,Mobiliado,Portaria,Metro
0,17/04/2023,https://loft.com.br/imovel/apartamento-rua-caure-tucuruvi-sao-paulo-2-quartos-50m2/100tsld,Condomínio Vila Verde - Rua Cauré - Tucuruvi - São Paulo - SP,Tucuruvi,R$ 325.000,3º andar,50 m²,R$ 6.500 /m²,2 quartos,sem suíte,1 vaga,1 Banheiro,Sem varanda,Não mobiliado,Portaria24h,Sem Metro próximo
1,17/04/2023,https://loft.com.br/imovel/apartamento-rua-do-bosque-bom-retiro-sao-paulo-2-quartos-72m2/1014uwp,Condomínio Edificio Palhinha - Rua do Bosque - Bom Retiro - São Paulo - SP,Bom Retiro,R$ 479.000,4º andar em diante,72 m²,R$ 6.652 /m²,2 quartos,sem suíte,Não - Vagas,1 Banheiro,Sem varanda,Não mobiliado,Sem info,Metro a 885m
2,17/04/2023,https://loft.com.br/imovel/apartamento-rua-jose-maria-lisboa-jardim-america-sao-paulo-3-quartos-137m2/101r6vc,Condomínio Anambé - Rua José Maria Lisboa - Jardim América - São Paulo - SP,Jardim América,R$ 1.450.000,4º andar em diante,137 m²,R$ 10.583 /m²,3 quartos,1 suíte,Não - Vagas,3 Banheiros,Sem varanda,Não mobiliado,Sem info,Metro a 702m
3,17/04/2023,https://loft.com.br/imovel/apartamento-avenida-doutor-cardoso-de-melo-vila-olimpia-sao-paulo-1-quarto-72m2/1024379,Condomínio Maxhaus Vila Olimpia - Avenida Doutor Cardoso de Melo - Vila Olímpia - São Paulo - SP,Vila Olímpia,R$ 1.400.000,4º andar em diante,72 m²,R$ 19.444 /m²,1 quarto,1 suíte,2 vagas,2 Banheiros,Varanda,Não mobiliado,Sem info,Sem Metro próximo
4,17/04/2023,https://loft.com.br/imovel/apartamento-rua-nunes-garcia-santana-sao-paulo-4-quartos-200m2/102ww3k,Condomínio Conjunto Santana Edificio Solar De Coimbra - Rua Nunes Garcia - Santana - São Paulo - SP,Santana,R$ 850.000,4º andar em diante,200 m²,R$ 4.250 /m²,4 quartos,1 suíte,3 vagas,4 Banheiros,Sem varanda,Não mobiliado,Sem info,Metro a 519m


### Drop

- Analisar se o Apto. teve variação no valor; 
- Verificar se houve duplicatas na rodada, caso não tenha, armazenar o histórico com a data em que rodou; 

##### Eliminando duplicatas 

In [ ]:
antes = df.shape[0]

df = df.drop_duplicates().reset_index(drop=True)
depois = df.shape[0]

print('Quantidade de linhas duplicadas: {}'.format(antes - depois))
print('Shape DataFrame atual: {}'.format(df.shape))

### Arrumando o tipo das variáveis

In [67]:
# Categoricas

# Mobiliado
df['Mobiliado'] = df['Mobiliado'].apply(lambda x: 'Não' if (x == 'Não mobiliado') else 
                                                              'Sim' if (x == 'Mobiliado') else x)

# Portaria
df['Portaria'] = df['Portaria'].apply(lambda x: '24h' if (x == 'Portaria24h') else 'Parcial' if (x == 'Portariaparcial') else 'Sem descrição')

# =================================================================

# Númericas

# Valor (R$)
df['Valor'] = df['Valor'].apply(lambda x: x.split('R$ ')[1])
df['Valor'] = df['Valor'].apply(lambda x: x.replace('.', '')).astype('int64')

# Valor_m2
df['Valor (m2)'] = df['Valor (m2)'].apply(lambda x: x.split('R$ ')[1].split(' /m²')[0])
df['Valor (m2)'] = df['Valor (m2)'].apply(lambda x: x.replace('.', '')).astype('int64')

# Tamanho (m2)
df['Tamanho (m2)'] = df['Tamanho (m2)'].apply(lambda x: x.split(' m²')[0]).astype('int64')

# Qtt_quartos
# Considerando que studio tem 1 quarto
df['Quartos'] = df['Quartos'].apply(lambda x: '1' if (x == 'studio') else x.split('qua')[0]).astype('int64')

# Suites
df['Suites'] = df['Suites'].apply(lambda x: '0' if (x == 'sem suíte') else x.split(' su')[0]).astype('int64')

# Garagem
df['Garagem'] = df['Garagem'].apply(lambda x: '0' if ((x == 'Não - Vagas') | (x == 'Sem info')) else x.split(' va')[0]).astype('int64')

# Banheiro
df['Banheiros'] = df['Banheiros'].apply(lambda x: x.split(' Ba')[0]).astype('int64')

# Varanda
df['Varanda'] = df['Varanda'].apply(lambda x: '0' if (x == 'Sem varanda') else '1' if (x == 'Varanda') else x.split(' Va')[0]).astype('int64')

# Metro
df['Metro'] = df['Metro'].apply(lambda x: 100000000 if (x == 'Sem Metro próximo') else x.split('Metro a ')[-1].replace('m', '')).astype('int64')

In [68]:
df.dtypes

Refdate         object
Link_Apto       object
Endereco        object
Bairro          object
Valor            int64
Andares         object
Tamanho (m2)     int64
Valor (m2)       int64
Quartos          int64
Suites           int64
Garagem          int64
Banheiros        int64
Varanda          int64
Mobiliado       object
Portaria        object
Metro            int64
dtype: object

### Lat Long

<ul>- API Geocoding, parece ser paga.</ul>
<ul><a href='https://developers.google.com/maps/documentation/geocoding/overview?hl=pt-br'>site</a></ul>
<ul><a href='https://www.youtube.com/watch?v=d1QGLwie9YU'>vídeo</a> <font color='yellow'> → assistir ao vídeo!</font></ul>

- Caso não tenha nenhuma API disponível, gratuita, ver a possíbilidade de fazer um webscraping no site; 

#### API GeoPY (Google Maps)

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

data = {'Id': ['1', '2', '3', '4'], 
        'Endereco': ['Rua Paracatu, 251 - Parque Imperial', 'Rua Beatriz, 226 - Vila Madalena', 'Av. Paulista, 1230 - Bela Vista', 'Av. Ibirapuera, 3103 - Indianópolis'], 
        'CEP': ['04302-020', '05445-040', '01310-100', '04029-902']}
        
df = pd.DataFrame(data)

df['End_CEP'] = df.apply(lambda x: x['Endereco'] + ' - São Paulo - SP' + ', ' + x['CEP'], axis=1)
df['End_Completo'] = df.apply(lambda x: x['Endereco'] + ' - São Paulo - SP', axis=1)
df['Latitude'] = df['Longitude'] = ''

df.head()

In [ ]:
locator = Nominatim(user_agent="myGeocoder")

for i in range(df.shape[0]):
  try:
    # location = locator.geocode(df.loc[i, 'End_Completo'],  timeout=10)
    location = locator.geocode(df.loc[i, 'End_CEP'],  timeout=5)
    df.loc[i, 'Latitude'] = location.latitude
    df.loc[i, 'Longitude'] = location.longitude
  except Exception as e : 
    print('O Id= {} deu erro'.format(df.loc[i, 'Id']))
    print(e)

df.head()

In [ ]:
locator = Nominatim(user_agent="myGeocoder")

location = locator.geocode('Rua Cauré - Tucuruvi - São Paulo - SP',  timeout=5)
print(location)

#### Base Feito "a mão"

In [ ]:
# Salvando os Bairros Disponíveis
df_bairro = pd.DataFrame(df['Bairro'].unique())
df_bairro.to_csv('./Bairros.csv', index=False, encoding= 'utf-8')
df_bairro.head()

In [ ]:
# Preenchido com os Dados de Lat Long → Verificar se existe alguma API
df_latlong = pd.read_csv('lat_long.csv', encoding='utf-8')
df_merge = pd.merge(df, df_latlong, how='left', on='Bairro')[['Refdate', 'Bairro', 'Lat', 'Long', 'Valor', 'Andares', 'Tamanho', 'Valor_m2', 'Qtt_quartos', 'Suites', 'Garagem', 'Banheiro', 'Varanda', 'Mobiliado', 'Portaria']]

# Ajustando Nome dos Bairros
dict = {'FreguesiadoÓ': 'Frequezia do Ó', 'Parquedo Carmo': 'Parque do Carmo', 'JardimÂngela': 'Jardim Ângela', 
        'Altoda Lapa': 'Alto da Lapa', 'Altode Pinheiros': 'Alto de Pinheiros', 'MoemaÍndios': 'Moema Índios'}

df_merge['Bairro'] = df_merge['Bairro'].apply(lambda x: dict[x] if (x in dict) else x)

# print('Linhas totais Lat_Long: {}'.format(df_latlong.shape[0]))
# print('Linhas totais DF Original: {}'.format(df.shape[0]))
# print('Linhas totais: {}'.format(df_merge.shape[0]))

df_merge.head()

### DataFrame Final

In [69]:
df.head()

,Refdate,Link_Apto,Endereco,Bairro,Valor,Andares,Tamanho (m2),Valor (m2),Quartos,Suites,Garagem,Banheiros,Varanda,Mobiliado,Portaria,Metro
0,17/04/2023,https://loft.com.br/imovel/apartamento-rua-caure-tucuruvi-sao-paulo-2-quartos-50m2/100tsld,Condomínio Vila Verde - Rua Cauré - Tucuruvi - São Paulo - SP,Tucuruvi,325000,3º andar,50,6500,2,0,1,1,0,Não,24h,100000000
1,17/04/2023,https://loft.com.br/imovel/apartamento-rua-do-bosque-bom-retiro-sao-paulo-2-quartos-72m2/1014uwp,Condomínio Edificio Palhinha - Rua do Bosque - Bom Retiro - São Paulo - SP,Bom Retiro,479000,4º andar em diante,72,6652,2,0,0,1,0,Não,Sem descrição,885
2,17/04/2023,https://loft.com.br/imovel/apartamento-rua-jose-maria-lisboa-jardim-america-sao-paulo-3-quartos-137m2/101r6vc,Condomínio Anambé - Rua José Maria Lisboa - Jardim América - São Paulo - SP,Jardim América,1450000,4º andar em diante,137,10583,3,1,0,3,0,Não,Sem descrição,702
3,17/04/2023,https://loft.com.br/imovel/apartamento-avenida-doutor-cardoso-de-melo-vila-olimpia-sao-paulo-1-quarto-72m2/1024379,Condomínio Maxhaus Vila Olimpia - Avenida Doutor Cardoso de Melo - Vila Olímpia - São Paulo - SP,Vila Olímpia,1400000,4º andar em diante,72,19444,1,1,2,2,1,Não,Sem descrição,100000000
4,17/04/2023,https://loft.com.br/imovel/apartamento-rua-nunes-garcia-santana-sao-paulo-4-quartos-200m2/102ww3k,Condomínio Conjunto Santana Edificio Solar De Coimbra - Rua Nunes Garcia - Santana - São Paulo - SP,Santana,850000,4º andar em diante,200,4250,4,1,3,4,0,Não,Sem descrição,519


In [70]:
df.to_csv('./final.csv', index=False, encoding= 'utf-8')